In [5]:
from sklearn.model_selection import GridSearchCV

from sklearn import model_selection  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss

In [14]:

# HAN YIKAI
feature4rf = ['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo',
              'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'indfall', 'cod_prov',
              'nomprov', 'ind_actividad_cliente', 'renta']
# feature4rf = ['age',
#               'antiguedad', 'renta']
feature4rf = ["ind_empleado", "pais_residencia", "sexo",
              "age", "ind_nuevo", "antiguedad", "nomprov",
              "segmento"]

file = './input/featured_data.csv'
df = pd.read_csv(file,  low_memory=False)
# df = df.sample(n=200000)

print(f'df.shape {df.shape}')


df.shape (49406, 44)


In [15]:
rm_cols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
           'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
           'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
           'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
           'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
           'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
           'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
           'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

targetcols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
              'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
              'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
              'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
              'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
              'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
              'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
              'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [16]:
def split_label(data):
    train_data = data.drop(rm_cols, axis=1)
    train_label = data[targetcols]

    return train_data, train_label



In [17]:

train_set, test_set = split_label(df)
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_set, test_set, test_size=0.2)

train_x_4_rf = train_x[feature4rf]
test_x_4_rf = test_x[feature4rf]



In [18]:
train_y['ind_nomina_ult1'].value_counts()

0.0    38133
1.0     1391
Name: ind_nomina_ult1, dtype: int64

In [19]:
train_y['ind_nomina_ult1'] = train_y['ind_nomina_ult1'].astype(np.int64)

In [20]:
train_y['ind_nomina_ult1'].dtype

dtype('int64')

In [21]:


"""
plot result fig to evaluate model performance 
"""


def plot_result(x, y, name='rf', save=False):
    plt.cla()
    plt.xlabel('number of min_samples_leaf', fontsize=12)
    plt.ylabel(name, fontsize=12)
    fig = sns.lineplot(x=x, y=y)
    if not save:
        plt.show()
    if save:
        pic = fig.get_figure()
        pic.savefig(f'{name}.png', dpi=150)


"""
select useful features with impact
"""


def select_features(model):
    features = []
    thresh_hold = 0.15
    importance = model.feature_importances_
    index_num = np.argsort(importance)[::-1]

    for f in range(train_x_4_rf.shape[1]):
        print("%2d) %-*s %f" % (f + 1, 30, feature4rf[index_num[f]], importance[index_num[f]]))
        if importance[index_num[f]] >= thresh_hold:
            features.append(feature4rf[index_num[f]])
    print(f'features selected {features}')

    return features


In [34]:
test_x_4_rf

,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,nomprov,segmento
27051,0,0,1,23.0,0,36.0,51,3.0
9611,0,0,0,52.0,0,12.0,12,2.0
49243,0,0,0,23.0,0,32.0,30,3.0
14325,0,0,1,32.0,0,37.0,24,3.0
13189,0,0,0,24.0,0,32.0,37,3.0
...,...,...,...,...,...,...,...,...
30170,0,0,0,22.0,0,12.0,28,3.0
13837,0,0,1,23.0,0,36.0,13,3.0
20971,0,0,1,27.0,0,47.0,6,3.0
24123,0,0,1,31.0,0,35.0,14,3.0


In [35]:


# n_estimators = list(range(10, 200, 10))
# max_depth = list(range(15, 60, 5))
# min_weight_fraction_leaf = list(range(0, 10, 2))
# param_grid = {'n_estimators': n_estimators,
#               'oob_score': [True],
#               # 'min_samples_split': [5],
#               # 'min_samples_leaf': [30],
#             #   'min_weight_fraction_leaf': min_weight_fraction_leaf,
#               'max_depth': [20]
#               }

# grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid,
#                            cv=3)
# clf = grid_search
clf = RandomForestClassifier(n_estimators=20,
                             max_depth=None,
                             n_jobs=-1,
                             random_state=2021,
                             # class_weight='balanced'
                             )
print('Training....')
# grid_search.fit(train_x_4_rf, train_y)
clf.fit(train_x_4_rf, train_y)
grid_search= clf
h_loss_l = []
acc_l = []
best_score_l = []
print('Predicting....')
predict_v = clf.predict(test_x_4_rf)

# print('acc:', accuracy_score(test_y, predict_v))

res = pd.DataFrame(predict_v)

h_loss = hamming_loss(test_y, predict_v)
print(f'hamming_loss is : {h_loss}')
# print(res.iloc[11])

# select features in first time, clf have been annotation
# selected_features = select_features(clf)

# mean_test_score = grid_search.cv_results_['mean_test_score']
# print(f'cv {mean_test_score}')
# print(f'type {type(mean_test_score)}')

# use grid search to determine the best params in this model
# print(f'best params {grid_search.best_params_}')
# print(f'best score {grid_search.best_score_}')
x_columns_indices = []
# plot_result(test_x_4_rf, predict_v)
# filter features
# x_selected = train_x[:, importances > threshold]
# print(x_selected)

# plot_result(n_estimators, mean_test_score, name='accuracy of Random forest')
# plot_result(n_estimators, l_hamming, name='hamming.png')


Training....
Predicting....


/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/pandas/core/internals/construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


ValueError: could not broadcast input array from shape (9882,1) into shape (9882,)

In [23]:
res.shape

(9882, 24)

In [24]:
def show_f1(y_true, y_pred):

    from sklearn.metrics import f1_score


    print(f1_score(y_true, y_pred, average='macro')) 
    print(f1_score(y_true, y_pred, average='micro')) 
    print(f1_score(y_true, y_pred, average='weighted'))
    print(f1_score(y_true, y_pred, average=None))


In [25]:
show_f1(test_y, predict_v)


0.1626588074566583
0.7487410440122824
0.6954454614295155
[0.         0.         0.93350369 0.         0.11386139 0.98203593
 0.42994242 0.         0.22429907 0.         0.         0.37430168
 0.32792485 0.         0.         0.         0.         0.03546099
 0.08727273 0.         0.         0.0875     0.08778626 0.21992238]


/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-

In [26]:
import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(clf.estimators_[0],
                                feature_names=feature4rf,
                                filled=True,
                                rounded=True)

graph = graphviz.Source(dot_data)
